In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np

url = 'https://www.bbc.com/korean/news-46562902'
r = requests.get(url)

soup = BeautifulSoup(r.text, "html.parser")

In [2]:
mr = soup.find(class_="story-body__introduction")

In [3]:
text = mr.get_text()

In [4]:
split_text = list(text)

In [5]:
spacing = []
for i in range(len(split_text)):
    if split_text[i] == ' ':
        spacing.append(([split_text[i-1], split_text[i+1]]))

In [6]:
spacing

[['늘', '밤'],
 [',', '쌍'],
 ['리', '유'],
 ['가', '쏟'],
 ['서', '2'],
 ['의', '가'],
 ['장', '화'],
 ['한', '우'],
 ['가', '밤'],
 ['에', '펼'],
 ['질', '것'],
 ['로', '보']]

In [7]:
split1 = split_text
df = pd.get_dummies(split1)

for j in range(44):
    if split_text[j] == ' ':
        print(j)
        split_text.pop(j)

In [8]:
split_text = list(filter((' ').__ne__, split_text))

In [9]:
split2 = split_text
df2 = pd.get_dummies(split2)

In [10]:
df2.values.shape

(47, 42)

In [11]:
xdata = df2.values

In [12]:
y = np.ones([xdata.shape[0], 1])

In [13]:
for k in range(len(spacing)):
    for l in range(len(split2)):
        if split2[l] == spacing[k][0]:
            if split2[l+1] == spacing[k][1]:
                print(l)
                y[l][0] = y[l][0]-1

1
3
8
12
17
23
25
28
32
36
39
42


In [14]:
xdata.shape

(47, 42)

In [15]:
import numpy as np
import tensorflow as tf
import datetime
sess = tf.Session()

x = xdata


y_ = np.zeros([xdata.shape[0], xdata.shape[0]-1])

y = np.concatenate((y, y_),axis = 1)


c_ = tf.zeros([xdata.shape[0],xdata.shape[0]])
h_ = tf.zeros([xdata.shape[0],xdata.shape[0]])


X = tf.placeholder(dtype=tf.float32, shape=[None, xdata.shape[1]])
Y = tf.placeholder(dtype=tf.float32, shape=[None, xdata.shape[0]])
W = tf.Variable(tf.random_normal([xdata.shape[0], 1]))*0.5
b = tf.Variable(tf.random_normal([1]))


#he
#el
#ll
#lo

seq_len = len(x) #len(x)cell 갯수, 인풋이 몇 덩어리인지
num_units = len(sess.run(c_))  #len(sess.run((c_)))# hiddenlayer


class lstm:
    def build(c, h):
        args = tf.concat((X,h), axis=1)
#        print(args)

        out_size = 4 * num_units
        proj_size = args.shape[-1]
#        print(out_size)
#        print(proj_size)

        weights = tf.ones([proj_size, out_size]) * 0.5
#        print(weights)


        out = tf.matmul(args, weights)
#        print(out)

        bias = tf.ones([out_size]) * 0.5
#        print(bias)

        concat = out + bias
#        print(concat)

        i, j, f, o = tf.split(concat, 4, 1)
#        print(i)
#        print(j)
#        print(f)
#        print(o)

        g = tf.tanh(j)
#        print(g)

        def sigmoid_array(x):
            return 1 / (1 + tf.exp(-x))

        forget_bias = 1.0

        sigmoid_f = sigmoid_array(f + forget_bias)
#        print(sigmoid_f)

        sigmoid_array(i) * g

        new_c = c * sigmoid_f + sigmoid_array(i) * g
#        print(new_c)

        new_h = tf.tanh(new_c) * sigmoid_array(o)
#        print(new_h)

#        print('\n new_h:',new_h)
#        print('\n new_c',new_c)

#        print(res[1].h)
#        print(res[1].c)

        return new_c, new_h

In [16]:
bx = x[::-1]

by = y[::-1]



bc_ = tf.zeros([xdata.shape[0],xdata.shape[0]])
bh_ = tf.zeros([xdata.shape[0],xdata.shape[0]])


bX = tf.placeholder(dtype=tf.float32, shape=[None, xdata.shape[1]])
bY = tf.placeholder(dtype=tf.float32, shape=[None, xdata.shape[0]])
bW = tf.Variable(tf.random_normal([xdata.shape[0], 1]))*0.5
bb = tf.Variable(tf.random_normal([1]))

class blstm:
    def build(c, h):
        args = tf.concat((X,h), axis=1)
#        print(args)

        out_size = 4 * num_units
        proj_size = args.shape[-1]
#        print(out_size)
#        print(proj_size)

        weights = tf.ones([proj_size, out_size]) * 0.5
#        print(weights)


        out = tf.matmul(args, weights)
#        print(out)

        bias = tf.ones([out_size]) * 0.5
#        print(bias)

        concat = out + bias
#        print(concat)

        i, j, f, o = tf.split(concat, 4, 1)
#        print(i)
#        print(j)
#        print(f)
#        print(o)

        g = tf.tanh(j)
#        print(g)

        def sigmoid_array(x):
            return 1 / (1 + tf.exp(-x))

        forget_bias = 1.0

        sigmoid_f = sigmoid_array(f + forget_bias)
#        print(sigmoid_f)

        sigmoid_array(i) * g

        new_bc = c * sigmoid_f + sigmoid_array(i) * g
#        print(new_c)

        new_bh = tf.tanh(new_bc) * sigmoid_array(o)
#        print(new_h)

#        print('\n new_h:',new_h)
#        print('\n new_c',new_c)

#        print(res[1].h)
#        print(res[1].c)

        return new_bc, new_bh

##########################flstm

In [17]:
ta_c = tf.TensorArray(size=seq_len, dtype=tf.float32)
ta_h = tf.TensorArray(size=seq_len, dtype=tf.float32)

In [18]:
def body(last_state, last_output, step, ta_c, ta_h):
    
    output = lstm.build(last_state, last_output)[0]
    state = lstm.build(last_state, last_output)[1]
    ta_c = ta_c.write(step, state)
    ta_h = ta_h.write(step, output)
    return state, output, tf.add(step, 1), ta_c, ta_h
    

timesteps = seq_len

steps = lambda a, b, step, c, d: tf.less(step, timesteps)

lstm_output, lstm_state, step, ta_c, ta_h = tf.while_loop(steps, body, (c_, h_, 0, ta_c, ta_h), parallel_iterations=20)

In [19]:
output = lstm_output
logits = tf.matmul(output, W) + b

In [20]:
with tf.name_scope('mean_square_error'):
    mean_square_error = tf.reduce_sum(tf.square(tf.subtract(Y, tf.unstack(logits, axis = 1))))
tf.summary.scalar('mean_square_error', mean_square_error)

<tf.Tensor 'mean_square_error_1:0' shape=() dtype=string>

In [21]:
optimizer = tf.train.AdamOptimizer(0.0003)
minimize = optimizer.minimize(mean_square_error)

In [22]:
with tf.name_scope('error'):
    with tf.name_scope('mistakes'):
        mistakes = tf.not_equal(Y, tf.round(tf.unstack(logits, axis = 1)))
    with tf.name_scope('error'):
        error = tf.reduce_mean(tf.cast(mistakes, tf.float32))
tf.summary.scalar('error', error)

<tf.Tensor 'error_1:0' shape=() dtype=string>

In [23]:
sess = tf.Session()
merged = tf.summary.merge_all()

date = str(datetime.datetime.now())

init_op = tf.global_variables_initializer()
sess.run(init_op)

In [24]:
epoch = 3000

for i in range(epoch):
    if (i + 1) % 100 == 0:
        summary, incorrect, mean_squ_err = sess.run([merged, error, mean_square_error], {X:x, Y:y})
        
        print('Epoch {:4d} | incorrect {: 3.1f}% | mean squ error {: 3.1f}'.format(i + 1, incorrect * 100, mean_squ_err))
    else:
        summary, acc = sess.run([merged, error], {X:x, Y:y})


    sess.run(minimize,{X:x, Y:y})

Epoch  100 | incorrect  100.0% | mean squ error  11097.0
Epoch  200 | incorrect  100.0% | mean squ error  6077.0
Epoch  300 | incorrect  98.4% | mean squ error  3064.3
Epoch  400 | incorrect  98.4% | mean squ error  1414.1
Epoch  500 | incorrect  98.4% | mean squ error  599.7
Epoch  600 | incorrect  1.6% | mean squ error  241.2
Epoch  700 | incorrect  1.6% | mean squ error  101.7
Epoch  800 | incorrect  1.6% | mean squ error  53.8
Epoch  900 | incorrect  1.6% | mean squ error  39.3
Epoch 1000 | incorrect  1.6% | mean squ error  35.5
Epoch 1100 | incorrect  1.6% | mean squ error  34.7
Epoch 1200 | incorrect  1.6% | mean squ error  34.5
Epoch 1300 | incorrect  1.6% | mean squ error  34.5
Epoch 1400 | incorrect  1.6% | mean squ error  34.4
Epoch 1500 | incorrect  1.6% | mean squ error  34.4
Epoch 1600 | incorrect  1.6% | mean squ error  34.4
Epoch 1700 | incorrect  1.6% | mean squ error  34.4
Epoch 1800 | incorrect  1.6% | mean squ error  34.4
Epoch 1900 | incorrect  1.6% | mean squ error

##################################

##################################back

In [25]:
bta_c = tf.TensorArray(size=seq_len, dtype=tf.float32)
bta_h = tf.TensorArray(size=seq_len, dtype=tf.float32)

In [26]:
def bbody(last_state, last_output, step, bta_c, bta_h):
    
    boutput = blstm.build(last_state, last_output)[0]
    bstate = blstm.build(last_state, last_output)[1]
    bta_c = bta_c.write(step, bstate)
    bta_h = bta_h.write(step, boutput)
    return bstate, boutput, tf.add(step, 1), bta_c, bta_h
    

timesteps = seq_len


steps = lambda a, b, step, c, d: tf.less(step, timesteps)

blstm_output, blstm_state, step, bta_c, bta_h = tf.while_loop(steps, bbody, (bc_, bh_, 0, bta_c, bta_h), parallel_iterations=20)

In [27]:
boutput = blstm_output
blogits = tf.matmul(boutput, W) + b

In [28]:
with tf.name_scope('mean_square_error'):
    mean_square_error = tf.reduce_sum(tf.square(tf.subtract(Y, tf.unstack(logits, axis = 1))))
tf.summary.scalar('mean_square_error', mean_square_error)

<tf.Tensor 'mean_square_error_3:0' shape=() dtype=string>

In [29]:
optimizer = tf.train.AdamOptimizer(0.0003)
minimize = optimizer.minimize(mean_square_error)

In [30]:
with tf.name_scope('error'):
    with tf.name_scope('mistakes'):
        mistakes = tf.not_equal(Y, tf.round(tf.unstack(logits, axis = 1)))
    with tf.name_scope('error'):
        error = tf.reduce_mean(tf.cast(mistakes, tf.float32))
tf.summary.scalar('error', error)

<tf.Tensor 'error_3:0' shape=() dtype=string>

In [31]:
sess = tf.Session()
merged = tf.summary.merge_all()

date = str(datetime.datetime.now())

init_op = tf.global_variables_initializer()
sess.run(init_op)

In [32]:
epoch = 3000

for i in range(epoch):
    if (i + 1) % 100 == 0:
        summary, incorrect, mean_squ_err = sess.run([merged, error, mean_square_error], {X:x, Y:y})
        
        print('Epoch {:4d} | incorrect {: 3.1f}% | mean squ error {: 3.1f}'.format(i + 1, incorrect * 100, mean_squ_err))
    else:
        summary, acc = sess.run([merged, error], {X:x, Y:y})


    sess.run(minimize,{X:x, Y:y})

Epoch  100 | incorrect  100.0% | mean squ error  66134.0
Epoch  200 | incorrect  100.0% | mean squ error  51298.1
Epoch  300 | incorrect  100.0% | mean squ error  39145.3
Epoch  400 | incorrect  100.0% | mean squ error  29341.6
Epoch  500 | incorrect  100.0% | mean squ error  21568.1
Epoch  600 | incorrect  100.0% | mean squ error  15520.9
Epoch  700 | incorrect  100.0% | mean squ error  10915.5
Epoch  800 | incorrect  100.0% | mean squ error  7489.0
Epoch  900 | incorrect  100.0% | mean squ error  5004.3
Epoch 1000 | incorrect  98.4% | mean squ error  3252.4
Epoch 1100 | incorrect  98.4% | mean squ error  2053.8
Epoch 1200 | incorrect  98.4% | mean squ error  1260.2
Epoch 1300 | incorrect  98.4% | mean squ error  752.7
Epoch 1400 | incorrect  1.6% | mean squ error  440.0
Epoch 1500 | incorrect  1.6% | mean squ error  254.6
Epoch 1600 | incorrect  1.6% | mean squ error  149.2
Epoch 1700 | incorrect  1.6% | mean squ error  91.7
Epoch 1800 | incorrect  1.6% | mean squ error  61.7
Epoch 1

########################################

In [33]:
fw = sess.run(tf.equal(sess.run(Y, feed_dict={X:x, Y:y}),sess.run(tf.round(tf.unstack(logits, axis = 1)),feed_dict={X:x, Y:y})))
fw = sess.run(tf.one_hot(fw, 1, axis=0))

In [34]:
bw = sess.run(tf.equal(sess.run(Y, feed_dict={X:bx, Y:by}),sess.run(tf.round(tf.unstack(blogits, axis = 1)),feed_dict={X:bx, Y:by})))
bw = sess.run(tf.one_hot(bw, 1, axis=0))

In [35]:
fw = fw[0]
fw = fw[:,[0]]

In [36]:
bw = bw[0]
bw = bw[:,[0]]

In [37]:
Bi_Lstm_Output = np.column_stack((fw,bw))

In [38]:
Bi_Lstm_Output

array([[1., 1.],
       [0., 1.],
       [1., 1.],
       [0., 1.],
       [1., 0.],
       [1., 1.],
       [1., 1.],
       [1., 0.],
       [0., 1.],
       [1., 1.],
       [1., 0.],
       [1., 1.],
       [0., 1.],
       [1., 1.],
       [1., 0.],
       [1., 1.],
       [1., 1.],
       [0., 1.],
       [1., 0.],
       [1., 1.],
       [1., 1.],
       [1., 0.],
       [1., 1.],
       [0., 0.],
       [1., 1.],
       [0., 1.],
       [1., 1.],
       [1., 1.],
       [0., 1.],
       [1., 0.],
       [1., 1.],
       [1., 1.],
       [0., 1.],
       [1., 1.],
       [1., 0.],
       [1., 1.],
       [0., 1.],
       [1., 1.],
       [1., 0.],
       [0., 1.],
       [1., 1.],
       [1., 1.],
       [0., 1.],
       [1., 0.],
       [1., 1.],
       [1., 0.],
       [1., 1.]], dtype=float32)

In [39]:
sentence = split2
s_output = []

for i in range(len(sentence)):
    s_output.append(sentence[i])
    if Bi_Lstm_Output[i][0] == 0:
        s_output.append(' ')

In [40]:
s_output

['오',
 '늘',
 ' ',
 '밤',
 ',',
 ' ',
 '쌍',
 '둥',
 '이',
 '자',
 '리',
 ' ',
 '유',
 '성',
 '우',
 '가',
 ' ',
 '쏟',
 '아',
 '지',
 '면',
 '서',
 ' ',
 '2',
 '0',
 '1',
 '8',
 '년',
 '의',
 ' ',
 '가',
 '장',
 ' ',
 '화',
 '려',
 '한',
 ' ',
 '우',
 '주',
 '쇼',
 '가',
 ' ',
 '밤',
 '하',
 '늘',
 '에',
 ' ',
 '펼',
 '쳐',
 '질',
 ' ',
 '것',
 '으',
 '로',
 ' ',
 '보',
 '인',
 '다',
 '.']

In [41]:
bs_output = []

for i in range(len(sentence)):
    bs_output.append(sentence[i])
    if Bi_Lstm_Output[-i-1][1] == 0:
        bs_output.append(' ')

In [42]:
bs_output

['오',
 '늘',
 ' ',
 '밤',
 ',',
 ' ',
 '쌍',
 '둥',
 '이',
 '자',
 '리',
 ' ',
 '유',
 '성',
 '우',
 '가',
 ' ',
 '쏟',
 '아',
 '지',
 '면',
 '서',
 ' ',
 '2',
 '0',
 '1',
 '8',
 '년',
 '의',
 ' ',
 '가',
 '장',
 ' ',
 '화',
 '려',
 '한',
 ' ',
 '우',
 '주',
 '쇼',
 '가',
 ' ',
 '밤',
 '하',
 '늘',
 '에',
 ' ',
 '펼',
 '쳐',
 '질',
 ' ',
 '것',
 '으',
 '로',
 ' ',
 '보',
 '인',
 '다',
 '.']